In [1]:

from geopandas.base import GeoPandasBase
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely import wkt



In [10]:
### get this done !!!
def get_nonitri_sum2(df):
    def cond(df):
        try:
            if df['no_nitri']==True:
                return df['PE']
            else:
                return 0
                
        except:
            if df['NITRIFIZIERUNG']==False:
                return df['EW60']
            else:
                return 0


    df['PE_nonitri']=df.apply(cond, axis=1)

    return df
 


def join_nospat(df, filename):
    shp = geopandas.read_file('DATA/shp_new/Oesterreich_BEV_VGD_LAM.shp')
    shp.KG_NR = shp.KG_NR.astype(int)
    shp.to_crs(epsg=4326, inplace=True)
    merged=pd.merge(shp, df, on='KG_NR')
    if len(df[~df.KG_NR.isin(merged.KG_NR)]) ==0:
        print('perfect merge')
    else:
        print('not all rows have been merged')
        not_merged=df[~df.KG_NR.isin(merged.KG_NR)]
        not_merged.to_excel('final/' +filename+'not_merged.xlsx')
    print()

    return merged

def extract_data_nospat(joined):
    """
    return df with one row per KG
    only summary columns like sum EW,  average values and so on

    insert column for each Tech_type sum ? like sum_SBR=5
    """
    joined=get_nonitri_sum2(joined)

    df1=joined.groupby(['KG_NR']).agg(lambda row: np.count_nonzero(row)).reset_index()
    df2=joined.groupby(['KG_NR']).sum().reset_index().loc[:,['KG_NR','PE','PE_nonitri']]
    try:
        df3=joined.groupby(['KG_NR']).mean().astype(int).reset_index().loc[:,['KG_NR','year']]
    except :
        df3=joined.groupby(['KG_NR']).mean().reset_index().loc[:,['KG_NR','year']].astype(int)

    merged=pd.merge(df2,df3, on='KG_NR')
    print(merged)
    extracted=df1.merge(merged,on='KG_NR')

    extracted.rename(columns={'PE_y':'sum_PE','year_y':'mean_year','MERIDIAN':'freq', 'PE_nonitri_y':'PE_nonitri' },inplace=True)
    extracted=extracted[['KG_NR','freq','before_reg', 'sum_PE','mean_year', 'no_nitri','PE_nonitri']]
    return extracted

def final_merge_nospat(extracted, BL_name):
    """
    return geojson with one row per KG
    contains geoinformation and ready to plot

    """

    shp = geopandas.read_file('DATA/shp_new/Oesterreich_BEV_VGD_LAM.shp')
    shp.KG_NR = shp.KG_NR.astype(int)
    shp.to_crs(epsg=4326, inplace=True)
    final=pd.merge(shp,extracted, on='KG_NR')

    print(final.geometry.is_valid.value_counts())

    if len(extracted)==len(final):
        print('perfect merge')
    else:
        print('not all rows have been merged')
        not_merged=extracted[~extracted.KG_NR.isin(final.KG_NR)]
        not_merged.to_excel('final/' +BL_name+'not_merged2.xlsx')
    print()


    #getting relative values
    #final['%no_nitri']=final.no_nitri/final.freq*100
    #final['%before_reg']=final.before_reg/final.freq*100
    #final['%PE_nonitri']=(final.PE_nonitri/final.sum_PE)*100

    #`final=standardize_format(final)


    with open('final/' + BL_name +'.geojson', 'w') as f:
        f.write(final.to_json())
    return final

In [3]:
data=pd.read_excel(r'C:\Users\fabrizio\Documents\repos\MSC\half-way/steyr.xlsx')

In [4]:
merged=join_nospat(data, 'steyr')

not all rows have been merged



In [41]:

merged['design']=np.where(merged.PE<50, 'small','medium')

In [46]:

merged[['PE','design']].sort_values('PE',ascending=False)

,PE,design
7428,300,medium
3014,200,medium
8599,200,medium
4612,200,medium
7405,200,medium
...,...,...
2313,0,small
6012,0,small
6013,0,small
2327,0,small


In [32]:
small=merged[merged.design=='small']
medium=merged[merged.design=='medium']

In [28]:
extracted=extract_data_nospat(merged)

      KG_NR   PE  PE_nonitri  year
0     60001   10          10  1966
1     60002    0           0  2005
2     60003   10           0  2004
3     60004   47          12  1992
4     60007   18          10  1982
...     ...  ...         ...   ...
1245  68262   75           0  2008
1246  68265   33           0  1986
1247  68266    0           0  1955
1248  68267  179           0  1908
1249  68268   16           0  1993

[1250 rows x 4 columns]


In [29]:
len(merged)
merged.PE.sum()

86895

In [30]:
extracted.freq.sum()
extracted.sum_PE.sum()

86895

In [ ]:
extracted

,KG_NR,freq,before_reg,sum_PE,mean_year,no_nitri,PE_nonitri,size
0,60001,1,1,10,1966,1,10,small
1,60002,1,0,0,2005,0,0,small
2,60003,1,0,10,2004,0,0,small
3,60004,4,1,47,1992,1,12,small
4,60007,3,2,18,1982,2,10,small
...,...,...,...,...,...,...,...,...
1509,68262,7,0,75,2008,0,0,small
1510,68265,3,1,33,1986,1,0,small
1511,68266,3,3,0,1955,3,0,small
1512,68267,21,1,179,1908,0,0,small


In [33]:
small=extract_data_nospat(small)
medium=extract_data_nospat(medium)

C:\Users\fabrizio\anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


      KG_NR   PE  PE_nonitri  year
0     60001   10          10  1966
1     60002    0           0  2005
2     60003   10           0  2004
3     60004   47          12  1992
4     60007   18          10  1982
...     ...  ...         ...   ...
1241  68262   75           0  2008
1242  68265   33           0  1986
1243  68266    0           0  1955
1244  68267  179           0  1908
1245  68268   16           0  1993

[1246 rows x 4 columns]
    KG_NR   PE  PE_nonitri  year
0   60013   60           0  2013
1   60102   50          50  1987
2   60217   60           0  2012
3   60301   75          75  1974
4   60303  170          50  1986
..    ...  ...         ...   ...
87  67319   60           0  2012
88  67510   50           0  2008
89  67611   90          90  1965
90  67613   50          50  1975
91  68025   50           0  1998

[92 rows x 4 columns]


C:\Users\fabrizio\anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [49]:
total=small.merge(medium, on='KG_NR',how='outer',suffixes=('_small','_medium'))

In [50]:
total.columns

Index(['KG_NR', 'freq_small', 'before_reg_small', 'sum_PE_small',
       'mean_year_small', 'no_nitri_small', 'PE_nonitri_small', 'freq_medium',
       'before_reg_medium', 'sum_PE_medium', 'mean_year_medium',
       'no_nitri_medium', 'PE_nonitri_medium'],
      dtype='object')

In [ ]:
x=x[['KG_NR', 'freq_small','sum_PE_small','no_nitri_small', 'PE_nonitri_small',
        'freq_medium', 'sum_PE_medium','no_nitri_medium', 'PE_nonitri_medium']]

In [35]:
x=x.fillna(0)
x['freq_tot']=x.freq_small+x.freq_medium
x['sum_PE_tot']=x.sum_PE_small+x.sum_PE_medium
x['no_nitri_tot']=x.no_nitri_small+x.no_nitri_medium
x['PE_nonitri_tot']=x.PE_nonitri_small+x.PE_nonitri_medium

In [ ]:
x=x[['KG_NR', 'freq_small','freq_medium','freq_tot','sum_PE_small','sum_PE_medium','sum_PE_tot',
    'no_nitri_small','no_nitri_medium','no_nitri_tot','PE_nonitri_small','PE_nonitri_medium','PE_nonitri_tot']]

In [37]:
x.freq_tot.sum()
#x.sum_PE_tot.sum()

10338.0

In [47]:
geo=final_merge_nospat(x, 'stey')

True    1250
dtype: int64
perfect merge



In [40]:
geo.sum_PE_tot.sum()

86895.0